In [3]:
tag2id, id2tag = {}, {}
word2id, id2word = {}, {}
for line in open('traindata.txt'):
    items = line.split('/')
    word, tag = items[0], items[1].rstrip()
    if word not in word2id:
        word2id[word] = len(word2id)
        id2word[len(id2word)] = word
    if tag not in tag2id:
        tag2id[tag] = len(tag2id)
        id2tag[len(id2tag)] = tag

M = len(word2id)  # the #. of words in dictionary
N = len(tag2id)   # the #. of tags in tag set

In [4]:
# 构建 Pi A B
import numpy as np

pi = np.zeros(N)        # 初始状态概率矩阵
A = np.zeros((N, M))    # 发射概率矩阵
B = np.zeros((N, N))    # 状态转移概率矩阵

# 计算矩阵中的对应数据出现的次数
prev_tag = ''
for line in open('data/traindata.txt'):
    items = line.split('/')
    wordId, tagId = word2id[items[0]], tag2id[items[1].rstrip()]
    if prev_tag == '':
        pi[tagId] += 1
        A[tagId][wordId] += 1
    else:
        A[tagId][wordId] += 1
        B[tag2id[prev_tag]][tagId] += 1
    if items[0] == '.':
        prev_tag = ''
    else:
        prev_tag = id2tag[tagId]

# normalize 将统计的个数化成概率
pi = pi / sum(pi)
for i in range(N):
    A[i] /= sum(A[i])
    B[i] /= sum(B[i])
# 到此为止，计算完了模型的所有参数：pi A B

In [5]:
def log(v):
    if v == 0:
        return np.log(v + 0.00000001)
    return np.log(v)

In [20]:
# viterbi algorithm 
def viterbi(x, pi, A, B):
    """
    x: user input string / sentence 
    pi: initial probability of tags 
    A: 给定tag,每个单词出现的概率(发射概率)
    B: tag之间的转移概率(转移概率)
    """
    x = [word2id[word] for word in x.split(' ')]
    T = len(x)
    
    dp = np.zeros((T, N))
    ptr = np.array([[0 for x in range(N)] for y in range(T)])
    for j in range(N):
        dp[0][j] = log(pi[j]) + log(A[j][x[0]])
    for i in range(1, T):
        for j in range(N):
            dp[i][j] = -9999
            for k in range(N):
                score = dp[i - 1][k] + log(B[k][j]) + log(A[j][x[i]])
                if score > dp[i][j]:
                    dp[i][j] = score
                    ptr[i][j] = k
    # decoding: 把最好的tag sequence打印出来
    best_seq = [0] * T  # best_seq = [1, 5, 2, 234,...]
    # step1: 找出对应于最后一个单词的词性
    best_seq[T - 1] = np.argmax(dp[T - 1])
    # step2: 通过从后到前的循环依次求出每个单词的词性
    for i in range(T-2, -1, -1):
        best_seq[i] = ptr[i + 1][best_seq[i + 1]]
    # 到目前为止，best_seq存放了对应于x的词性序列
    for i in range(len(best_seq)):
        print(id2tag[best_seq[i]])
    

In [21]:
x = 'Social Security number , passport .'
viterbi(x, pi, A, B)

NNP
NNP
NN
,
NN
.
